In [3]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

Reusing TensorBoard on port 6006 (pid 15032), started 23:20:36 ago. (Use '!kill 15032' to kill it.)

In [4]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '22_3'

In [5]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [12]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 190
SIZE = 32

In [7]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [8]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [9]:
import csv

embeddings = {}
with open("../inception_resnet_v2_files/embeddings-cifar10-normalization.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        filename = row[0]
        values = [float(val) for val in row[1:]]
        embeddings[filename] = np.array(values)

In [10]:
images_rgb = {}
imagespath = '../../datasets/cifar10/train'
imagespathval = '../../datasets/cifar10/val'
all_files = [os.path.join(imagespath, file) for file in os.listdir(imagespath) if os.path.isfile(
            os.path.join(imagespath, file))]
all_files2 = [os.path.join(imagespathval, file) for file in os.listdir(imagespathval) if os.path.isfile(
            os.path.join(imagespathval, file))]
all_files = all_files + all_files2
for image in all_files:
    rgbfilename = os.path.basename(image)
    img = Image.open(image).convert("RGB")
    images_rgb[rgbfilename] = img

In [11]:
print(len(embeddings))
print(embeddings["abandoned_ship_s_000004.png"][:20])
print(len(images_rgb))
print(images_rgb["abandoned_ship_s_000004.png"])

60000
[0.21317568 0.16253434 0.26581229 0.29878058 0.33441289 0.24510603
 0.51277979 0.18380144 0.1670414  0.15279668 0.10542652 0.20540888
 0.20522403 0.19407258 0.17829575 0.11919297 0.2119326  0.24213714
 0.23753528 0.18780443]
60000
<PIL.Image.Image image mode=RGB size=32x32 at 0x1C8FD038100>


In [13]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
                transforms.RandomVerticalFlip(),  
                transforms.RandomRotation(10),
                transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05)
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        itemfilename = os.path.basename(self.paths[index])
        img = images_rgb[itemfilename]
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        embd = embeddings[itemfilename]
        embd = torch.from_numpy(embd).unsqueeze(0).float()
        return img_gray, img_ab, embd
    
    def __len__(self):
        return len(self.paths)

In [14]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [15]:
class FusionLAyer(nn.Module):
    def __init__(self):
        super().__init__()
        pass

    def forward(self, imgs, embds):
        embds = torch.reshape(embds, (embds.shape[0], embds.shape[2]))
        reshaped_shape = (imgs.shape[0], embds.shape[1], imgs.shape[2], imgs.shape[3])
        #shapetosave = (embds.shape[0], imgs.shape[2] * imgs.shape[3], embds.shape[1])
        embds = embds.repeat(1,imgs.shape[2] * imgs.shape[3]) #shape (128, 64000)
        #embds = torch.reshape(embds, shapetosave)
        #embds = torch.reshape(embds, reshaped_shape)
        embds = torch.reshape(embds, reshaped_shape)
        cos = torch.cat((imgs, embds), dim=1)
        return cos

In [16]:
kernel_size=3
stride_en=1
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 256
p1 = .25

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.conv1 = nn.Conv2d(1, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.conv2 = nn.Conv2d(channels_base, channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)

        self.convtrans1 = nn.ConvTranspose2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(channels_base, channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base)
        self.batchnorm2 = nn.BatchNorm2d(channels_base * 2)
        self.batchnorm3 = nn.BatchNorm2d(channels_base)
        self.batchnorm4 = nn.BatchNorm2d(channels_base // 2)
        self.batchnorm5 = nn.BatchNorm2d(2)
        self.fusion = FusionLAyer()
        self.after_fusion = nn.Sequential(
            nn.Conv2d(1512,512, kernel_size=1),
            nn.ReLU()
        )
    def forward(self, input, embds):
        # encoder
        x = F.leaky_relu(self.batchnorm1(self.conv1(input)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = y = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        
        # fusion
        x = self.fusion(x, embds)
        x = self.after_fusion(x)

        # decoder
        x = F.leaky_relu(self.batchnorm3(self.convtrans1(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.leaky_relu(self.batchnorm4(self.convtrans2(x + y)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.leaky_relu(self.batchnorm5(self.convtrans3(x + input)), negative_slope=0.1)

        return x

In [17]:
model = Autoencoder()

In [18]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [19]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [20]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [36]:
if use_gpu: 
    from torchsummary import summary
    summary(model, [(1, SIZE, SIZE), (1, 1000)])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 256, 32, 32]           2,560
       BatchNorm2d-2          [-1, 256, 32, 32]             512
            Conv2d-3          [-1, 512, 16, 16]       1,180,160
       BatchNorm2d-4          [-1, 512, 16, 16]           1,024
       FusionLAyer-5           [-1, 1512, 8, 8]               0
            Conv2d-6            [-1, 512, 8, 8]         774,656
              ReLU-7            [-1, 512, 8, 8]               0
   ConvTranspose2d-8            [-1, 256, 8, 8]       1,179,904
       BatchNorm2d-9            [-1, 256, 8, 8]             512
  ConvTranspose2d-10          [-1, 128, 16, 16]         295,040
      BatchNorm2d-11          [-1, 128, 16, 16]             256
  ConvTranspose2d-12            [-1, 2, 32, 32]           2,306
      BatchNorm2d-13            [-1, 2, 32, 32]               4


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [21]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [22]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab, embd in val_loader:
        if use_gpu: 
            gray, ab, embd = gray.cuda(), ab.cuda(), embd.cuda()

        # Run model and record loss
        output_ab = model(gray, embd) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [23]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab, embd in train_loader:
        if use_gpu: 
            gray, ab, embd = gray.cuda(), ab.cuda(), embd.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray, embd) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [24]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience and epoch >= 100:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.00391515 (0.03801172), PSNR 24.07252121 (16.87089740), SSIM 0.66131264 (0.47322552)
Finished training epoch 0
Validate: MSE 0.00393993 (0.00368657), PSNR 24.04511452 (24.39277297), SSIM 0.73579979 (0.76105766)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.00297600 (0.00272551), PSNR 25.26367188 (25.66437306), SSIM 0.76981807 (0.76599596)
Finished training epoch 1
Validate: MSE 0.00238210 (0.00267880), PSNR 26.23039627 (25.79377950), SSIM 0.74541461 (0.76241601)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.00255516 (0.00250684), PSNR 25.92582130 (26.02367646), SSIM 0.80467421 (0.76163762)
Finished training epoch 2
Validate: MSE 0.00230264 (0.00244546), PSNR 26.37773514 (26.17657536), SSIM 0.75221699 (0.76831451)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.00233573 (0.00243368), PSNR 26.31577492 (26.15230386), SSIM 0.76125062 (0.76413744)
Finished training epoch 3
Validate: MSE 0.00232053 (0.0

<Figure size 640x480 with 0 Axes>

In [25]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [26]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.00135395 (0.00141788), PSNR 28.68397522 (28.53345046), SSIM 0.78236592 (0.79302593)
Finished validation.


<Figure size 640x480 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()